# Application #

## Required Librairies

In [1]:
#import the libraries
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import tkinter as tk
from tkinter import ttk

import pymysql as mysql
import csv
import os
import time
import sys


## Database Connection ##

In [2]:
hostname = "127.0.0.1"
username = "michaeld_aoop"
password = "1704206"
database = "michaeld_fyp"

connection = mysql.connect(host=hostname, user=username, passwd=password, db=database, autocommit=True,local_infile=1)


## GUI ##

In [ ]:
#font size for all text and buttons
LARGE_FONT=("Verdana",12)
#font size for application title
EXTRA_LARGE_FONT=("Verdana",20)

#the main application window, this is what will load
class MyApplication(tk.Tk):
    ##the init method. this intialises everything that is needed
    def __init__(self, *args, **kwargs):
        ##call the init on itself
        tk.Tk.__init__(self, *args, **kwargs)
        #create container for the frames
        container = tk.Frame(self)
        #display on the screen
        container.pack(side="top", fill="both", expand = True)
        #configure container rows
        container.grid_rowconfigure(0, weight=1)
        #configure container columns
        container.grid_columnconfigure(0, weight=1)
        #create an empty list for the frames
        self.frames = {}
        
        #for all frames in my gui forms
        for F in (MainMenu, InsertEditMenu, InsertMenu, EditMenu, RecommendationMenu, RecommendationMainMenu, EmotionMenu, 
                  InsertForm, InsertEmotionForm, AssignEmotionForm,
                 RecEArtistForm,RecEDurationForm, RecEGenreForm, RecEReleasedForm,
                  RecommendArtistForm, RecommendGenreForm, RecommendDurationForm, RecommendReleasedForm, ViewForm,
                 EditEmotionsForm,EditForm):
            #make a frame for the forms
            frame = F(container, self)
            
            #set the applications frames to be these frames
            self.frames[F] = frame
            
            #make it so that the form centralises
            frame.grid(row=0, column=0, sticky="nsew")
        #show the main menu frame
        self.show_frame(MainMenu)
    #create a method to show the frames
    def show_frame(self, cont):
        
        #set frame to be the frame shown in the parameter
        frame = self.frames[cont]
        #show the frame
        frame.tkraise()

        
#the recommendation form for date released + emotion
class RecEReleasedForm(tk.Frame):
    
    #initialise the form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        
        #create arrays so that we can use them to get correct form details
        ids=[]
        names=[]
        artists=[]
        releases=[]
        genres=[]
        durations=[]
        emotions=[]
        
        #create method to rite all songs from database to csv file, pass the connection to the file
        def select_songse(con):
            #create a cursor so that we can execute mysql queries
            cursor1=con.cursor()
            #execute query to select required info from database.
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions")
            #set row variable to the result of the query
            row=cursor1.fetchall()
            #either create or open csv file with following name for editing
            with open('songsEmotions.csv', 'w', newline= '') as f:
                #say we want to write to the file
                a = csv.writer(f, delimiter=',')
                #head the file same way as in database
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write the required rows
                a.writerows(row)
        
        #create function to select the recommendations for the emotion and date released
        def select_recER(con, songname, released, emotion):
            #create cursor
            cursor1=con.cursor()
            #execute required query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions WHERE song_name LIKE \"%%"+songname+"%%\" OR date_released LIKE \"%%"+released+"%%\" OR emotion LIKE \"%%"+emotion+"%%\"")
            #fetch the required results and assign to row variable
            row=cursor1.fetchall()
            
            #create or open file
            with open('recEmReleased.csv', 'w', newline= '') as f:
                #say we want to write to them
                a = csv.writer(f, delimiter=',')
                #head the rows
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write rows from result
                a.writerows(row)
            #open file that was written to
            with open('recEmReleased.csv') as csvDataFile:
                #say that we are reading from the file
                csvReader = csv.reader(csvDataFile)
                #for every row in the file
                for row in csvReader:
                    #add to arrays
                    ids.append(row[0])
                    names.append(row[1])
                    artists.append(row[2])
                    releases.append(row[3])
                    genres.append(row[4])
                    durations.append(row[5])
                    emotions.append(row[6])
        #select the songs
        select_songse(connection)
        
        #initialise needed variables
        songName=""
        songEmotion=""
        releaseYear=""
        releaseMonth=""
        releaseDay=""
        
        #label for title
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: MAKE RECOMMENDATION BASED ON RELEASE DATE & EMOTION", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        
        #label for song name input
        lblSongName = tk.Label(self, text="Insert Song Name: ", font=LARGE_FONT)
        lblSongName.pack()
        
        #song name input
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        
        #label for song emotion input
        lblSongEmotion = tk.Label(self, text="Insert Song Emotion: ", font=LARGE_FONT)
        lblSongEmotion.pack()
                            
        #song emotion input
        txtSongEmotion=tk.Entry(self, textvariable=songEmotion, font=LARGE_FONT)
        txtSongEmotion.pack()
                            
        #label for release day
        lblReleaseDay = tk.Label(self, text="Insert Release Day in format DD: ", font=LARGE_FONT)
        lblReleaseDay.pack()
                            
        #release day input
        txtReleaseDay=tk.Entry(self, textvariable=releaseDay, font=LARGE_FONT)
        txtReleaseDay.pack()
        
        #label for release month
        lblReleaseMonth = tk.Label(self, text="Insert Release Month in format MM: ", font=LARGE_FONT)
        lblReleaseMonth.pack()
        
        #release month input
        txtReleaseMonth=tk.Entry(self, textvariable=releaseDay, font=LARGE_FONT)
        txtReleaseMonth.pack()
        
        #label for release year
        lblReleaseYear = tk.Label(self, text="Insert Release Year in format YYYY: ", font=LARGE_FONT)
        lblReleaseYear.pack()
        
        #release year input
        txtReleaseYear=tk.Entry(self, textvariable=releaseDay, font=LARGE_FONT)
        txtReleaseYear.pack()
        
        #listbox to display result
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        
        #function to get song name from selected index
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]

        #function to get song artist from index
        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        #function to get genre from index
        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        #function to get song duration from index
        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        #function to get release date from index
        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        #function to get the emotion  from the index
        def get_emotion_from_index(index):
            df["emotion"]=df["emotion"].astype(str)
            return df[df.song_id == index]["emotion"].values[0]
        
        #get the song index from the name
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songsEmotions.csv")
        
        #convert values to a list                    
        songs=df.values.tolist()
                            
        #for every song in the songs list                    
        for song in songs:

            #add to rusult listbox                
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"Assigned Emotion: "+get_emotion_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        
        #function that recommends songs                    
        def recommendEmotionReleasedG():
            #set required variables                
            songName=""
            songEmotion=""
            releaseDay=""
            releaseMonth=""
            releaseYear=""
            ###### helper functions. Use them when needed #######
            #needed functions
            def get_name_from_index(index):
                #if (index==0)
                #    index = 1
                
                #            
                return df[df.song_id == index]["song_name"].values[0]

            #needed functions              
            def get_artist_from_index(index):
                
                return df[df.song_id == index]["artist"].values[0]

            #needed functions                
            def get_genre_from_index(index):
                
                return df[df.song_id == index]["song_genre"].values[0]

            #needed functions                
            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            #needed functions               
            def get_released_from_index(index):
                
                return df[df.song_id == index]["date_released"].values[0]
            
            #needed functions               
            def get_emotion_from_index(index):
                
                return df[df.song_id == index]["emotion"].values[0]
            
            #needed functions               
            def get_index_from_name(title):
                #df["song_id"]=int(df["song_id"])
                return df[df.song_name == title]["song_id"].values[0]

                            
            ##################################################
            ##Step 1: Read CSV File
                            
            #set variables to inputs                
            songName=txtSongName.get()
            releaseDay=txtReleaseDay.get()
            releaseMonth=txtReleaseMonth.get()
            releaseDate=txtReleaseYear.get()
            releaseDate=releaseYear+"-"+releaseMonth+"-"+releaseDay
            songEmotion=txtSongEmotion.get()
            select_recER(connection, songName, releaseDate, songEmotion)
            df = pd.read_csv("recEmReleased.csv")
            
            ##Step 2: Select Features
            features = ['date_released', 'emotion']
                            
            ##Step 3: Create a column in Data Frame which combines all selected features
            for feature in features:
                df[feature] = df[feature].fillna('')
            
            #combine the features that are needed                
            def combine_features(row):
                return row['date_released'] + " " + row['emotion']
                            
                            
            #if(len(df)!=0):
            #
            df["combined_features"] = df.apply(combine_features, axis=1)
            
                            
            ##Step 4: Create count matrix from this new combined column
            cv = CountVectorizer()
            #transform into a matrix
            count_matrix = cv.fit_transform(df["combined_features"])

            ##Step 5: Compute the Cosine Similarity based on the count_matrix
            cosine_sim = cosine_similarity(count_matrix)
                            
            #set song name variable to song name text                
            songName=txtSongName.get()
                            
            #set this to the song name variable                
            song_user_likes = songName

            ## Step 6: Get index of this song from its title
            song_index = get_index_from_name(song_user_likes)
            
            #similar song list is the cosine_simularity of the song index
            similar_songs = list(enumerate(cosine_sim[song_index]))
            ## Step 7: Get a list of similar songs in descending order of similarity score
            sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)[1:]

            ## Step 8: Print titles of first 50 songs
            i=1
            #clear listbox
            lbxResult.delete(0, tk.END)
            #loop through songs
            for song in sorted_similar_songs:
                #insert song position and song details into list box
                lbxResult.insert(tk.END, "Song Position:"+str(i))
                lbxResult.insert(tk.END,"Song ID: "+ ids[i])
                lbxResult.insert(tk.END,"Song Name: "+names[i])
                lbxResult.insert(tk.END,"Artist: "+artists[i])
                lbxResult.insert(tk.END,"Genre: "+genres[i])
                lbxResult.insert(tk.END,"Duration: "+durations[i])
                lbxResult.insert(tk.END,"Date Released: "+releases[i])
                lbxResult.insert(tk.END,"Assigned Emotion: "+emotions[i])
                lbxResult.insert(tk.END,"----------------------------------")
                #increase counter
                i=i+1
                #if counter is more than 50, stop looping through
                if i>50:
                    break
        #button to make the recommendation
        btnMakeRecommendation = tk.Button(self, text="Make Recommendation", font=LARGE_FONT, command=recommendEmotionReleasedG)
        btnMakeRecommendation.pack()

        #button to go back to emotion menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(EmotionMenu))
        btnBack.pack() 
        

#form for recommendation with release date(no emotions)
class RecommendReleasedForm(tk.Frame):
    #initialise method
    def __init__(self, parent, controller):
        #initialise the form
        tk.Frame.__init__(self,parent)
        #arrays needed
        ids=[]
        names=[]
        artists=[]
        releases=[]
        genres=[]
        durations=[]
        emotions=[]
        #select songs and put them in csv file
        def select_songs(con):
            #cursor
            cursor1=con.cursor()
            #execute query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs")
            #fetch results
            row=cursor1.fetchall()
            #open file
            with open('songs.csv', 'w', newline= '') as f:
                #writer
                a = csv.writer(f, delimiter=',')
                #write headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs"])
                #write rows
                a.writerows(row)
        #recommendation function        
        def select_recR(con, songname, released):
            #cursor
            cursor1=con.cursor()
            #execute query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs WHERE song_name LIKE \"%%"+songname+"%%\" OR date_released LIKE \"%%"+released+"%%\"")
            #fetch rows
            row=cursor1.fetchall()
            #open file
            with open('songsReleased.csv', 'w', newline= '') as f:
                #writer
                a = csv.writer(f, delimiter=',')
                #write headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs"])
                #write the rows
                a.writerows(row)
            #open file
            with open('songsReleased.csv') as csvDataFile:
                #read csv files
                csvReader = csv.reader(csvDataFile)
                #for every row
                for row in csvReader:
                    #add to arrays
                    ids.append(row[0])
                    names.append(row[1])
                    artists.append(row[2])
                    releases.append(row[3])
                    genres.append(row[4])
                    durations.append(row[5])
        #select the songs
        select_songs(connection)
        #create required variables
        songName=""
        songEmotion=""
        releaseYear=""
        releaseMonth=""
        releaseDay=""
        
        #title
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: MAKE RECOMMENDATION BASED ON RELEASE DATE", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        
        #label for song
        lblSongName = tk.Label(self, text="Insert Song Name: ", font=LARGE_FONT)
        lblSongName.pack()
        
        #input for song
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        
        
        #label for release day
        lblReleaseDay = tk.Label(self, text="Insert Release Day in format DD: ", font=LARGE_FONT)
        lblReleaseDay.pack()
        
        #input for release day
        txtReleaseDay=tk.Entry(self, textvariable=releaseDay, font=LARGE_FONT)
        txtReleaseDay.pack()
        
        #label for release month
        lblReleaseMonth = tk.Label(self, text="Insert Release Day in format MM: ", font=LARGE_FONT)
        lblReleaseMonth.pack()
        
        #input for release month
        txtReleaseMonth=tk.Entry(self, textvariable=releaseDay, font=LARGE_FONT)
        txtReleaseMonth.pack()
        
        #label for release year
        lblReleaseYear = tk.Label(self, text="Insert Release Day in format YYYY: ", font=LARGE_FONT)
        lblReleaseYear.pack()
        
        #input for release year
        txtReleaseYear=tk.Entry(self, textvariable=releaseDay, font=LARGE_FONT)
        txtReleaseYear.pack()
        
        #listbox for result
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        
        
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]
                            
                          
        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        
        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        
        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        
        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        
        
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songs.csv")
        
        #convert songs to list
        songs=df.values.tolist()
        #for each item
        for song in songs:
            #insert values in list box
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        #recommend function
        def recommendReleasedG():
            #intialise required variables
            songName=""
            releaseDay=""
            releaseMonth=""
            releaseYear=""
            ###### helper functions. Use them when needed #######
            
            def get_name_from_index(index):
                #if (index==0)
                #    index = 1
                #
                return df[df.song_id == index]["song_name"].values[0]
                            
            
            def get_artist_from_index(index):
                
                return df[df.song_id == index]["artist"].values[0]
                            
            
            def get_genre_from_index(index):
                
                return df[df.song_id == index]["song_genre"].values[0]
            
            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]
            
            def get_released_from_index(index):
                
                return df[df.song_id == index]["date_released"].values[0]
            
            
            def get_index_from_name(title):
                #df["song_id"]=int(df["song_id"])
                return df[df.song_name == title]["song_id"].values[0]

            ##################################################
            ##Step 1: Read CSV File
            songName=txtSongName.get()
            releaseDay=txtReleaseDay.get()
            releaseMonth=txtReleaseMonth.get()
            releaseDate=txtReleaseYear.get()
            releaseDate=releaseYear+"-"+releaseMonth+"-"+releaseDay
            select_recR(connection, songName, releaseDate)
            df = pd.read_csv("songsReleased.csv")
            
            ##Step 2: Select Features
            features = ['date_released']
            ##Step 3: Create a column in Data Frame which combines all selected features
            for feature in features:
                df[feature] = df[feature].fillna('')
            def combine_features(row):
                return row['date_released']
            #if(len(df)!=0):
            df["combined_features"] = df.apply(combine_features, axis=1)
            
            ##Step 4: Create count matrix from this new combined column
            cv = CountVectorizer()
            #the matrix
            count_matrix = cv.fit_transform(df["combined_features"])

            ##Step 5: Compute the Cosine Similarity based on the count_matrix
            cosine_sim = cosine_similarity(count_matrix)
            #variable for the song name set to song name input
            songName=txtSongName.get()
            song_user_likes = songName

            ## Step 6: Get index of this song from its title
            song_index = get_index_from_name(song_user_likes)
            
            similar_songs = list(enumerate(cosine_sim[song_index]))
            ## Step 7: Get a list of similar songs in descending order of similarity score
            sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)[1:]

            ## Step 8: Print titles of first 50 songs
            i=1
            lbxResult.delete(0, tk.END)
            #for everything in the songs list
            for song in sorted_similar_songs:
                #insert everything in the list box
                lbxResult.insert(tk.END, "Song Position:"+str(i))
                lbxResult.insert(tk.END,"Song ID: "+ ids[i])
                lbxResult.insert(tk.END,"Song Name: "+names[i])
                lbxResult.insert(tk.END,"Artist: "+artists[i])
                lbxResult.insert(tk.END,"Genre: "+genres[i])
                lbxResult.insert(tk.END,"Duration: "+durations[i])
                lbxResult.insert(tk.END,"Date Released: "+releases[i])
                lbxResult.insert(tk.END,"----------------------------------")
                
                i=i+1
                if i>50:
                    break
        #button to make recommendation
        btnMakeRecommendation = tk.Button(self, text="Make Recommendation", font=LARGE_FONT, command=recommendReleasedG)
        btnMakeRecommendation.pack()

        #button to go back
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendationMenu))
        btnBack.pack() 

#gui for recommending artist
class RecommendArtistForm(tk.Frame):
    #init method and intialising the form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        
        #arrays needed for application
        ids=[]
        names=[]
        artists=[]
        releases=[]
        genres=[]
        durations=[]
        emotions=[]
        #select required songs
        def select_songs(con):
            #create cursor to execute queries
            cursor1=con.cursor()
            #execute the select query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs")
            #fetch the results
            row=cursor1.fetchall()
            #open the songs file to edit
            with open('songs.csv', 'w', newline= '') as f:
                #say we want to write to the file
                a = csv.writer(f, delimiter=',')
                #write headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs"])
                #write the rquired rows
                a.writerows(row)
        #recommendation for the artist. stores into the relevant file        
        def select_recA(con, songname, artist):
            #create cursor for executing queries
            cursor1=con.cursor()
            #execute the query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs WHERE song_name LIKE \"%%"+songname+"%%\" OR artist LIKE \"%%"+artist+"%%\"")
            #fetch the required info
            row=cursor1.fetchall()
            #open the file for recommendations
            with open('songsArtist.csv', 'w', newline= '') as f:
                #say that we want to write to the file
                a = csv.writer(f, delimiter=',')
                #write the headings 
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs"])
                #write the recommendations
                a.writerows(row)
            #open recommendation file    
            with open('songsArtist.csv') as csvDataFile:
                #say we want to retrieve data from the file
                csvReader = csv.reader(csvDataFile)
                #for every row in the file            
                for row in csvReader:
                    #add to arrays
                    ids.append(row[0])
                    names.append(row[1])
                    artists.append(row[2])
                    releases.append(row[3])
                    genres.append(row[4])
                    durations.append(row[5])
        #select the songs
        select_songs(connection)
        #empty variables. they need to be initialised
        songName=""
        songEmotion=""
        releaseYear=""
        releaseMonth=""
        releaseDay=""
        
        #title of form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: MAKE RECOMMENDATION BASED ON ARTIST", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #label for song name
        lblSongName = tk.Label(self, text="Insert Song Name: ", font=LARGE_FONT)
        lblSongName.pack()
        #input for song name
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        
        
        #label for song artist
        lblSongArtist = tk.Label(self, text="Insert Artist: ", font=LARGE_FONT)
        lblSongArtist.pack()
        #input for song artist
        txtSongArtist=tk.Entry(self, textvariable=releaseDay, font=LARGE_FONT)
        txtSongArtist.pack()
        
        
        #listbox for result
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]
        
        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        
        
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songs.csv")
        
        songs=df.values.tolist()
        #loop through songs and display them in listbox
        for song in songs:
                            
            #display
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        #function that recommends the artist
        def recommendArtistG():
            #init variables                
            songName=""
            songArtist=""
            ###### helper functions. Use them when needed #######
            def get_name_from_index(index):
                #if (index==0)
                #    index = 1
                
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                
                return df[df.song_id == index]["date_released"].values[0]
            
           
            
            def get_index_from_name(title):
                #df["song_id"]=int(df["song_id"])
                return df[df.song_name == title]["song_id"].values[0]

            ##################################################
            ##Step 1: Read CSV File
            songName=txtSongName.get()
            #set the required variables
            songEmotion=txtSongArtist.get()
            select_recA(connection, songName, songArtist)
            #open the file with recommendataions
            df = pd.read_csv("songsArtist.csv")
            
            ##Step 2: Select Feature
            features = ['artist']
            ##Step 3: Create a column in Data Frame which combines all selected features
            for feature in features:
                df[feature] = df[feature].fillna('')
            def combine_features(row):
                return row['artist']
            #if(len(df)!=0):
            df["combined_features"] = df.apply(combine_features, axis=1)
            
            ##Step 4: Create count matrix from this new combined column
            cv = CountVectorizer()
            #create a matrix
            count_matrix = cv.fit_transform(df["combined_features"])

            ##Step 5: Compute the Cosine Similarity based on the count_matrix
            cosine_sim = cosine_similarity(count_matrix)
            songName=txtSongName.get()
            song_user_likes = songName

            ## Step 6: Get index of this song from its title
            song_index = get_index_from_name(song_user_likes)
            
            similar_songs = list(enumerate(cosine_sim[song_index]))
            ## Step 7: Get a list of similar songs in descending order of similarity score
            sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)[1:]

            ## Step 8: Print titles of first 50 songs
            i=1
            #clear the list box
            lbxResult.delete(0, tk.END)
            #make it so that we add every song
            for song in sorted_similar_songs:
                #insert details into listbox
                lbxResult.insert(tk.END, "Song Position:"+str(i))
                lbxResult.insert(tk.END,"Song ID: "+ ids[i])
                lbxResult.insert(tk.END,"Song Name: "+names[i])
                lbxResult.insert(tk.END,"Artist: "+artists[i])
                lbxResult.insert(tk.END,"Genre: "+genres[i])
                lbxResult.insert(tk.END,"Duration: "+durations[i])
                lbxResult.insert(tk.END,"Date Released: "+releases[i])
                lbxResult.insert(tk.END,"----------------------------------")

                i=i+1
                if i>50:
                    break
        #button to trigger the recommedation function
        btnMakeRecommendation = tk.Button(self, text="Make Recommendation", font=LARGE_FONT, command=recommendArtistG)
        btnMakeRecommendation.pack()

        #button to go back to the recommendation menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendationMenu))
        btnBack.pack() 
#form to recommend the genre        
class RecommendGenreForm(tk.Frame):
    #init method to intialise the form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #initialise the arrays
        ids=[]
        names=[]
        artists=[]
        releases=[]
        genres=[]
        durations=[]
        emotions=[]
        #select required songs
        def select_songs(con):
            #cursor to create the queries
            cursor1=con.cursor()
            #execute the query from database
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs")
            #fetch required rows
            row=cursor1.fetchall()
            #open the songs file
            with open('songs.csv', 'w', newline= '') as f:
                #write to the files
                a = csv.writer(f, delimiter=',')
                #write the headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs"])
                #write the rows
                a.writerows(row)
        #recommend info for genre        
        def select_recG(con, songname, genre):
            #create cursor 
            cursor1=con.cursor()
            #execute queries
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs WHERE song_name LIKE \"%%"+songname+"%%\" OR song_genre LIKE \"%%"+genre+"%%\"")
            #fetch results
            row=cursor1.fetchall()
            #open song genre file
            with open('songsGenre.csv', 'w', newline= '') as f:
                #say we want to write to the file
                a = csv.writer(f, delimiter=',')
                #write headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs"])
                #write the required rows
                a.writerows(row)
            #open recommendations file
            with open('songsGenre.csv') as csvDataFile:
                #say we want to retrieve data from the file
                csvReader = csv.reader(csvDataFile)
                #for every row
                for row in csvReader:
                    #add to id
                    ids.append(row[0])
                    names.append(row[1])
                    artists.append(row[2])
                    releases.append(row[3])
                    genres.append(row[4])
                    durations.append(row[5])
        #select the songs
        select_songs(connection)
        #initialise variables
        songName=""
        songGenre=""
        
        #the title of the form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: MAKE RECOMMENDATION BASED GENRE", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #label for the song name input
        lblSongName = tk.Label(self, text="Insert Song Name: ", font=LARGE_FONT)
        lblSongName.pack()
        #input for song name
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        #label for the song genre
        lblSongGenre = tk.Label(self, text="Insert Song Genre: ", font=LARGE_FONT)
        lblSongGenre.pack()
        #input for the song genre
        txtSongGenre=tk.Entry(self, textvariable=songGenre, font=LARGE_FONT)
        txtSongGenre.pack()
        
        
        
        
        #the result
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        #functions needed to get relevant info
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]

        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songs.csv")
        #convert to list
        songs=df.values.tolist()
        for song in songs:
            #load the required info in the listbox
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        #function to recommend the genre
        def recommendGenreG():
            #initialise required variables
            songName=""
            songGenre=""
            ###### helper functions. Use them when needed #######
            #required functions to retrieve relevant info
            def get_name_from_index(index):
                #if (index==0)
                #    index = 1
                
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                
                return df[df.song_id == index]["date_released"].values[0]
            
           
            
            def get_index_from_name(title):
                #df["song_id"]=int(df["song_id"])
                return df[df.song_name == title]["song_id"].values[0]

            ##################################################
            ##Step 1: Read CSV File
            #get the song name from the text
            songName=txtSongName.get()
            #get song genre from input
            songGenre=txtSongGenre.get()
            #make the recommendation file
            select_recG(connection, songName, songGenre)
            #read the recommendation file
            df = pd.read_csv("songsGenre.csv")
            
            ##Step 2: Select Features
            features = ['song_genre']
            ##Step 3: Create a column in Data Frame which combines all selected features
            for feature in features:
                df[feature] = df[feature].fillna('')
            def combine_features(row):
                return row['song_genre']
            #if(len(df)!=0):
            df["combined_features"] = df.apply(combine_features, axis=1)
            
            ##Step 4: Create count matrix from this new combined column
            cv = CountVectorizer()

            count_matrix = cv.fit_transform(df["combined_features"])

            ##Step 5: Compute the Cosine Similarity based on the count_matrix
            cosine_sim = cosine_similarity(count_matrix)
            songName=txtSongName.get()
            song_user_likes = songName

            ## Step 6: Get index of this song from its title
            song_index = get_index_from_name(song_user_likes)
            
            similar_songs = list(enumerate(cosine_sim[song_index]))
            ## Step 7: Get a list of similar songs in descending order of similarity score
            sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)[1:]

            ## Step 8: Print titles of first 50 songs
            i=1
            #clear list
            lbxResult.delete(0, tk.END)
            #insert required info from list
            for song in sorted_similar_songs:
                
                lbxResult.insert(tk.END, "Song Position:"+str(i))
                lbxResult.insert(tk.END,"Song ID: "+ ids[i])
                lbxResult.insert(tk.END,"Song Name: "+names[i])
                lbxResult.insert(tk.END,"Artist: "+artists[i])
                lbxResult.insert(tk.END,"Genre: "+genres[i])
                lbxResult.insert(tk.END,"Duration: "+durations[i])
                lbxResult.insert(tk.END,"Date Released: "+releases[i])
                lbxResult.insert(tk.END,"----------------------------------")

                i=i+1
                if i>50:
                    break
        #button to call recommendation file
        btnMakeRecommendation = tk.Button(self, text="Make Recommendation", font=LARGE_FONT, command=recommendGenreG)
        btnMakeRecommendation.pack()

        #button to go back to recommendation menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendationMenu))
        btnBack.pack() 
#form for recommending the duration        
class RecommendDurationForm(tk.Frame):
    #function to initialise the form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #create required arrays
        ids=[]
        names=[]
        artists=[]
        releases=[]
        genres=[]
        durations=[]
        emotions=[]
        #make the song file and select them
        def select_songs(con):
            #create cursor for queries
            cursor1=con.cursor()
            #execute selection
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs")
            #store results
            row=cursor1.fetchall()
            #open the song file that is needed
            with open('songs.csv', 'w', newline= '') as f:
                #say we want to write to file
                a = csv.writer(f, delimiter=',')
                #write the headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs"])
                #write rows of song
                a.writerows(row)
        #make recommendation file        
        def select_recD(con, songname, duration):
            #create required cursor
            cursor1=con.cursor()
            #execute required query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs WHERE song_name LIKE \"%%"+songname+"%%\" OR song_length_secs rlike '[0-9]*"+duration+ "[0-9]*'")
            #store result of query
            row=cursor1.fetchall()
            #open the recommendation file
            with open('songsDuration.csv', 'w', newline= '') as f:
                #say we are writing to the file
                a = csv.writer(f, delimiter=',')
                #write the headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs"])
                #write the rows
                a.writerows(row)
            #open csv file again    
            with open('songsDuration.csv') as csvDataFile:
                #say we are retrieving data from the file
                csvReader = csv.reader(csvDataFile)
                #for every row in the file
                for row in csvReader:
                    #add to relevant arrays
                    ids.append(row[0])
                    names.append(row[1])
                    artists.append(row[2])
                    releases.append(row[3])
                    genres.append(row[4])
                    durations.append(row[5])
        #select the files    
        select_songs(connection)
        #initialise variables
        songName=""
        songDuration=""
        
        #title of application and form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: MAKE RECOMMENDATION BASED ON SONG DURATION", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #label for song name input
        lblSongName = tk.Label(self, text="Insert Song Name: ", font=LARGE_FONT)
        lblSongName.pack()
        #input for song name
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        #label for song duration input
        lblSongDuration = tk.Label(self, text="Insert Song Duration: ", font=LARGE_FONT)
        lblSongDuration.pack()
        #input for song duration
        txtSongDuration=tk.Entry(self, textvariable=songDuration, font=LARGE_FONT)
        txtSongDuration.pack()
       
        #listbox for results
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        #functions for getting required info
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]

        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        def get_emotion_from_index(index):
            df["emotion"]=df["emotion"].astype(str)
            return df[df.song_id == index]["emotion"].values[0]
        
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songs.csv")
        #convert to readable list
        songs=df.values.tolist()
        #for every row in file
        for song in songs:
            #display the result
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        #function to recommend the function
        def recommendDurationG():
            songName=""
            songDuration=""
            ###### helper functions. Use them when needed #######
            #get required functions
            def get_name_from_index(index):
                #if (index==0)
                #    index = 1
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                
                return df[df.song_id == index]["date_released"].values[0]
            
            def get_emotion_from_index(index):
                
                return df[df.song_id == index]["emotion"].values[0]
            
            def get_index_from_name(title):
                #df["song_id"]=int(df["song_id"])
                return df[df.song_name == title]["song_id"].values[0]

            ##################################################
            ##Step 1: Read CSV File
            songName=txtSongName.get()
            songDuration=txtSongDuration.get()
            select_recD(connection, songName, songDuration)
            df = pd.read_csv("songsDuration.csv")
            
            ##Step 2: Select Features
            features = ['song_length_secs']
            ##Step 3: Create a column in Data Frame which combines all selected features
            for feature in features:
                df[feature] = df[feature].fillna('')
            
            def combine_features(row): 
                return str(row['song_length_secs'])
            #if(len(df)!=0):
            df["combined_features"] = df.apply(combine_features, axis=1)
            
            ##Step 4: Create count matrix from this new combined column
            cv = CountVectorizer()
            #create the matrix needed
            count_matrix = cv.fit_transform(df["combined_features"])

            ##Step 5: Compute the Cosine Similarity based on the count_matrix
            cosine_sim = cosine_similarity(count_matrix)
            #set required variable
            song_user_likes = songName

            ## Step 6: Get index of this song from its title
            song_index = get_index_from_name(song_user_likes)
            #use cosine simularity to make a similar songs list
            similar_songs = list(enumerate(cosine_sim[song_index]))
            ## Step 7: Get a list of similar songs in descending order of similarity score
            sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)[1:]

            ## Step 8: Print titles of first 50 songs
            i=1
            #clear listbox so that songs arent doubled
            lbxResult.delete(0, tk.END)
            #for every song
            for song in sorted_similar_songs:
                #insert the required info
                lbxResult.insert(tk.END, "Song Position:"+str(i))
                lbxResult.insert(tk.END,"Song ID: "+ ids[i])
                lbxResult.insert(tk.END,"Song Name: "+names[i])
                lbxResult.insert(tk.END,"Artist: "+artists[i])
                lbxResult.insert(tk.END,"Genre: "+genres[i])
                lbxResult.insert(tk.END,"Duration: "+durations[i])
                lbxResult.insert(tk.END,"Date Released: "+releases[i])
                lbxResult.insert(tk.END,"----------------------------------")

                i=i+1
                if i>50:
                    break
        #button to trigger recommendation function
        btnMakeRecommendation = tk.Button(self, text="Make Recommendation", font=LARGE_FONT, command=recommendDurationG)
        btnMakeRecommendation.pack()

        #button to go back to recommendation menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendationMenu))
        btnBack.pack() 
        
#recommend genre and emotion form    
class RecEGenreForm(tk.Frame):
    
    #initialise the form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #initialise arrays
        ids=[]
        names=[]
        artists=[]
        releases=[]
        genres=[]
        durations=[]
        emotions=[]
        #function to select songs and emotions
        def select_songse(con):
            #create cursor for queries
            cursor1=con.cursor()
            #execute the queries
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions")
            #store the results
            row=cursor1.fetchall()
            #create or open filre for songs and emotions
            with open('songsEmotions.csv', 'w', newline= '') as f:
                #say that we want to write to file
                a = csv.writer(f, delimiter=',')
                #write the required headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write the rows
                a.writerows(row)
        #recommendation selection for emotion+genre
        def select_recEG(con, songname, genre, emotion):
            #create the cursor for executing queries
            cursor1=con.cursor()
            #execute the query and select required info
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions WHERE song_name LIKE \"%%"+songname+"%%\" OR song_genre LIKE \"%%"+genre+"%%\" OR emotion LIKE \"%%"+emotion+"%%\"")
            #store in the result
            row=cursor1.fetchall()
            #open recommendation file
            with open('recEmGenre.csv', 'w', newline= '') as f:
                #say we want to write to file
                a = csv.writer(f, delimiter=',')
                #write the headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write the rows
                a.writerows(row)
            #open the recommendation file
            with open('recEmGenre.csv') as csvDataFile:
                #say we want to retieve data from file
                csvReader = csv.reader(csvDataFile)
                #for every row in the reader
                for row in csvReader:
                    #insert into arrays
                    ids.append(row[0])
                    names.append(row[1])
                    artists.append(row[2])
                    releases.append(row[3])
                    genres.append(row[4])
                    durations.append(row[5])
                    emotions.append(row[6])
        #select rquired songs
        select_songse(connection)
        #set required variables
        songName=""
        songEmotion=""
        songGenre=""
        
        #title of application form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: MAKE RECOMMENDATION BASED ON GENRE & EMOTION", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
                            
        #label for song name input
        lblSongName = tk.Label(self, text="Insert Song Name: ", font=LARGE_FONT)
        lblSongName.pack()
                            
        #input for song name
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
                            
        #label for song emotion
        lblSongEmotion = tk.Label(self, text="Insert Song Emotion: ", font=LARGE_FONT)
        lblSongEmotion.pack()
                            
        #input for song emotion
        txtSongEmotion=tk.Entry(self, textvariable=songEmotion, font=LARGE_FONT)
        txtSongEmotion.pack()
                            
        #label for song genre
        lblSongGenre = tk.Label(self, text="Insert Genre: ", font=LARGE_FONT)
        lblSongGenre.pack()
                            
        #input for song genre
        txtSongGenre=tk.Entry(self, textvariable=songGenre, font=LARGE_FONT)
        txtSongGenre.pack()
                            
        #listbox for result
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
                            
        #get functions for required functions
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]

        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        def get_emotion_from_index(index):
            df["emotion"]=df["emotion"].astype(str)
            return df[df.song_id == index]["emotion"].values[0]
        
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songsEmotions.csv")
        #convert dataframe to list
        songs=df.values.tolist()
        #for everysong in the list
        for song in songs:
            #listbox for the result
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"Assigned Emotion: "+get_emotion_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        
        
        #recommendation function
        def recommendEmotionGenreG():
            ###### helper functions. Use them when needed #######
            #functions required for functions
            def get_name_from_index(index):
                df['song_name']=df['song_name'].astype(str)
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                df['artist']=df['artist'].astype(str)
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                df['song_genre']=df['song_genre'].astype(str)
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                df['date_released']=df['date_released'].astype(str)
                return df[df.song_id == index]["date_released"].values[0]
            
            def get_emotion_from_index(index):
                df["emotion"]=df["emotion"].astype(str)
                return df[df.song_id == index]["emotion"].values[0]
            
            def get_index_from_name(title):
                return df[df.song_name == title]["song_id"].values[0]

            ##################################################
            ##Step 1: Read CSV File
            #set variable to inputs
            songName=txtSongName.get()
            songEmotion=txtSongEmotion.get()
            songGenre=txtSongGenre.get()
                            #recommendation selection
            select_recEG(connection,songName,songGenre,songEmotion)
            #read csv file
            df = pd.read_csv("recEmGenre.csv")
            
            ##Step 2: Select Features
            features = ['song_genre', 'emotion']
            ##Step 3: Create a column in Data Frame which combines all selected features
            for feature in features:
                df[feature] = df[feature].fillna('')
            def combine_features(row):
                return row['song_genre'] + " " + row['emotion']
            #combine the features
            df["combined_features"] = df.apply(combine_features, axis=1)
            
            ##Step 4: Create count matrix from this new combined column
            cv = CountVectorizer()

            count_matrix = cv.fit_transform(df["combined_features"])

            ##Step 5: Compute the Cosine Similarity based on the count_matrix
            cosine_sim = cosine_similarity(count_matrix)
            #song name is equal to input
            songName=txtSongName.get()
            #where to get list data from
            song_user_likes = songName

            ## Step 6: Get index of this song from its title
            song_index = get_index_from_name(song_user_likes)

            similar_songs = list(enumerate(cosine_sim[song_index]))
            ## Step 7: Get a list of similar songs in descending order of similarity score
            sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)[1:]
            
            ## Step 8: Print titles of first 50 songs
            i=1
            lbxResult.delete(0, tk.END)
            #for every song
            for song in sorted_similar_songs:
                #insert data into listbox
                lbxResult.insert(tk.END, "Song Position:"+str(i))
                lbxResult.insert(tk.END,"Song ID: "+ ids[i])
                lbxResult.insert(tk.END,"Song Name: "+names[i])
                lbxResult.insert(tk.END,"Artist: "+artists[i])
                lbxResult.insert(tk.END,"Genre: "+genres[i])
                lbxResult.insert(tk.END,"Duration: "+durations[i])
                lbxResult.insert(tk.END,"Date Released: "+releases[i])
                lbxResult.insert(tk.END,"----------------------------------")

                i=i+1
                if i>50:
                    break
        #button to trigger 
        btnMakeRecommendation = tk.Button(self, text="Make Recommendation", font=LARGE_FONT, command=recommendEmotionGenreG)
        btnMakeRecommendation.pack()
        #button to go back to recommendation menu for emotions
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(EmotionMenu))
        btnBack.pack() 
        
        
#recommend duration and emotion        
class RecEDurationForm(tk.Frame):
    #initialise forms
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #set required arrays
        ids=[]
        names=[]
        artists=[]
        releases=[]
        genres=[]
        durations=[]
        emotions=[]
        #select songs from database and put in csv file
        def select_songse(con):
            #create cursor so that we can ex
            cursor1=con.cursor()
            #execute the select query to select song data from database
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions")
            #store result into variable
            row=cursor1.fetchall()
            #open the song emotion file
            with open('songsEmotions.csv', 'w', newline= '') as f:
                #say we are writing to the file
                a = csv.writer(f, delimiter=',')
                #write the headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write the rows
                a.writerows(row)
        #open the recommmendation file to make recommendation        
        def select_recED(con, songname, duration, emotion):
            #create cursor for queries
            cursor1=con.cursor()
            #execute the query                
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions WHERE song_name LIKE \"%%"+songname+"%%\" OR song_length_secs LIKE \"%%"+duration+"%%\" OR emotion LIKE \"%%"+emotion+"%%\"")
            #store the result
            row=cursor1.fetchall()
            #open recommendation file
            with open('recEmDuration.csv', 'w', newline= '') as f:
                #say we are writing to the file
                a = csv.writer(f, delimiter=',')
                #write the headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write the rows
                a.writerows(row)
            #open recommendation file again    
            with open('recEmDuration.csv') as csvDataFile:
                #say we are reading from file
                csvReader = csv.reader(csvDataFile)
                #for every record in file
                for row in csvReader:
                    #add relevent info to relevant arrays
                    ids.append(row[0])
                    names.append(row[1])
                    artists.append(row[2])
                    releases.append(row[3])
                    genres.append(row[4])
                    durations.append(row[5])
                    emotions.append(row[6])
        #select required songs
        select_songse(connection)
        
        #set required variables
        songName=""
        songDuration=""
        songEmotion=""
        
        
        #label for title
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: MAKE RECOMMENDATION BASED ON SONG DURATION & EMOTION", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #label for song name input
        lblSongName = tk.Label(self, text="Insert Song Name: ", font=LARGE_FONT)
        lblSongName.pack()
        #input for song name
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        #label for song duration
        lblSongDuration = tk.Label(self, text="Insert Song Duration: ", font=LARGE_FONT)
        lblSongDuration.pack()
        #input for song duration
        txtSongDuration=tk.Entry(self, textvariable=songDuration, font=LARGE_FONT)
        txtSongDuration.pack()
        #label for song emotion
        lblSongEmotion = tk.Label(self, text="Insert Song Emotion: ", font=LARGE_FONT)
        lblSongEmotion.pack()
        #input for song duration
        txtSongEmotion=tk.Entry(self, textvariable=songEmotion, font=LARGE_FONT)
        txtSongEmotion.pack()
        #listbox for result
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        #required functions
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]

        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        def get_emotion_from_index(index):
            df["emotion"]=df["emotion"].astype(str)
            return df[df.song_id == index]["emotion"].values[0]
        
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songsEmotions.csv")
        #convert dataframe to list
        songs=df.values.tolist()
        #for very item in the list
        for song in songs:
            #insert items into listbox
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"Assigned Emotion: "+get_emotion_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
            
        #recommendation function for emotion plus duration    
        def recommendEmotionDurationG():
            ###### helper functions. Use them when needed #######
            #required functions for the different columns of database table
            def get_name_from_index(index):
                df['song_name']=df['song_name'].astype(str)
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                df['artist']=df['artist'].astype(str)
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                df['song_genre']=df['song_genre'].astype(str)
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                df['date_released']=df['date_released'].astype(str)
                return df[df.song_id == index]["date_released"].values[0]
            
            def get_emotion_from_index(index):
                df["emotion"]=df["emotion"].astype(str)
                return df[df.song_id == index]["emotion"].values[0]
            
            def get_index_from_name(title):
                return df[df.song_name == title]["song_id"].values[0]

            ##################################################
            ##Step 1: Read CSV File
            #set variable to user input
            songName=txtSongName.get()
            songDuration=txtSongDuration.get()
            songEmotion=txtSongEmotion.get()
            select_recED(connection, songName, songDuration, songEmotion)
            df = pd.read_csv("recEmDuration.csv")

            ##Step 2: Select Features
            features = ['song_length_secs', 'emotion']
            ##Step 3: Create a column in Data Frame which combines all selected features
            for feature in features:
                df[feature] = df[feature].fillna('')
            def combine_features(row):
                return str(row['song_length_secs']) + " " + row['emotion']

            df["combined_features"] = df.apply(combine_features, axis=1)

            ##Step 4: Create count matrix from this new combined column
            cv = CountVectorizer()

            count_matrix = cv.fit_transform(df["combined_features"])

            ##Step 5: Compute the Cosine Similarity based on the count_matrix
            cosine_sim = cosine_similarity(count_matrix)
            songName=txtSongName.get()
            song_user_likes = songName

            ## Step 6: Get index of this song from its title
            song_index = get_index_from_name(song_user_likes)

            similar_songs = list(enumerate(cosine_sim[song_index]))
            ## Step 7: Get a list of similar songs in descending order of similarity score
            sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)[1:]

            ## Step 8: Print titles of first 50 songs
            i=1
            #clear the result
            lbxResult.delete(0, tk.END)
            #for every item in the lsit of similar songs
            for song in sorted_similar_songs:
                #insert into the list box
                lbxResult.insert(tk.END, "Song Position:"+str(i))
                lbxResult.insert(tk.END,"Song ID: "+ ids[i])
                lbxResult.insert(tk.END,"Song Name: "+names[i])
                lbxResult.insert(tk.END,"Artist: "+artists[i])
                lbxResult.insert(tk.END,"Genre: "+genres[i])
                lbxResult.insert(tk.END,"Duration: "+durations[i])
                lbxResult.insert(tk.END,"Date Released: "+releases[i])
                lbxResult.insert(tk.END,"----------------------------------")

                i=i+1
                if i>50:
                    break
        #recommendation button
        btnMakeRecommendation = tk.Button(self, text="Make Recommendation", font=LARGE_FONT, command=recommendEmotionDurationG)
        btnMakeRecommendation.pack()

        #button to go back to emotion menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(EmotionMenu))
        btnBack.pack() 
        
     
#recommend the artist + emotion        
class RecEArtistForm(tk.Frame):
    
   #initialise everythinh
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #create needed arrays for columns in field
        ids=[]
        names=[]
        artists=[]
        releases=[]
        genres=[]
        durations=[]
        emotions=[]
        #function to select the songs
        def select_songse(con):
            #create the cursor needed
            cursor1=con.cursor()
            #execute the select query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions")
            #store result
            row=cursor1.fetchall()
            #open the song file
            with open('songsEmotions.csv', 'w', newline= '') as f:
                #say we are writing to the file
                a = csv.writer(f, delimiter=',')
                #write the headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write the rows
                a.writerows(row)
        #select recommendation
        def select_recEA(con, songname, artist, emotion):
            #create required cursor
            cursor1=con.cursor()
            #execute query to select required info
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions WHERE song_name LIKE \"%%"+songname+"%%\" OR artist LIKE \"%%"+artist+"%%\" OR emotion LIKE \"%%"+emotion+"%%\"")
            #store result
            row=cursor1.fetchall()
            #open recommendation file
            with open('recEmArtist.csv', 'w', newline= '') as f:
                #say we are writing to file
                a = csv.writer(f, delimiter=',')
                #write headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write the rows
                a.writerows(row)
            #open recommendation file again
            with open('recEmArtist.csv') as csvDataFile:
                #say we are reading from the file
                csvReader = csv.reader(csvDataFile)
                #for every row in the files
                for row in csvReader:
                    #add to arrays
                    ids.append(row[0])
                    names.append(row[1])
                    artists.append(row[2])
                    releases.append(row[3])
                    genres.append(row[4])
                    durations.append(row[5])
                    emotions.append(row[6])
        #select required songs
        select_songse(connection)
        #initialise variables
        songName=""
        songArtist=""
        songEmotion=""
        
        #title of application
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: MAKE RECOMMENDATION BASED ON ARTIST & EMOTION", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #label for song input
        lblSongName = tk.Label(self, text="Insert Song Name: ", font=LARGE_FONT)
        lblSongName.pack()
        #input for song name
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        #label for song artist
        lblSongArtist = tk.Label(self, text="Insert Song Artist: ", font=LARGE_FONT)
        lblSongArtist.pack()
        #input for song artist
        txtSongArtist=tk.Entry(self, textvariable=songArtist, font=LARGE_FONT)
        txtSongArtist.pack()
        #label for song emotion
        lblSongEmotion = tk.Label(self, text="Insert Song Emotion: ", font=LARGE_FONT)
        lblSongEmotion.pack()
        #input for song emotion
        txtSongEmotion=tk.Entry(self, textvariable=songEmotion, font=LARGE_FONT)
        txtSongEmotion.pack()
        
        #listbox for displaying results
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        #functions to get required info
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]

        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        def get_emotion_from_index(index):
            df["emotion"]=df["emotion"].astype(str)
            return df[df.song_id == index]["emotion"].values[0]
        
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songsEmotions.csv")
        #convert dataframe to list
        songs=df.values.tolist()
        #for every song in list
        for song in songs:
            #display song info and separator
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"Assigned Emotion: "+get_emotion_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        
        
        
        #function used to recommend
        def recommendEmotionArtistG():
            #select songs
            select_songse(connection)
            ###### helper functions. Use them when needed #######
            def get_name_from_index(index):
                df['song_name']=df['song_name'].astype(str)
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                df['artist']=df['artist'].astype(str)
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                df['song_genre']=df['song_genre'].astype(str)
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                df['date_released']=df['date_released'].astype(str)
                return df[df.song_id == index]["date_released"].values[0]
            
            def get_emotion_from_index(index):
                df["emotion"]=df["emotion"].astype(str)
                return df[df.song_id == index]["emotion"].values[0]
            
            def get_index_from_name(title):
                return df[df.song_name == title]["song_id"].values[0]

            ##################################################
            ##Step 1: Read CSV File
            #set song name to user input
            songName=txtSongName.get()
            songArtist=txtSongArtist.get()
            songEmotion=txtSongEmotion.get()
            #select records from recommendation file
            select_recEA(connection, songName, songArtist, songEmotion)
            #read from recommendation file
            df = pd.read_csv("recEmArtist.csv")
            
            ##Step 2: Select Features
            features = ['artist', 'emotion']
            ##Step 3: Create a column in Data Frame which combines all selected features
            for feature in features:
                df[feature] = df[feature].fillna('')
            def combine_features(row):
                return row['artist'] + " " + row['emotion']
            #combine the features
            df["combined_features"] = df.apply(combine_features, axis=1)
            ##Step 4: Create count matrix from this new combined column
            cv = CountVectorizer()

            count_matrix = cv.fit_transform(df["combined_features"])

            ##Step 5: Compute the Cosine Similarity based on the count_matrix
            cosine_sim = cosine_similarity(count_matrix)
            songName=txtSongName.get()
            song_user_likes = songName

            ## Step 6: Get index of this song from its title
            song_index = get_index_from_name(song_user_likes)

            similar_songs = list(enumerate(cosine_sim[song_index]))
            ## Step 7: Get a list of similar songs in descending order of similarity score
            sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)[1:]

            ## Step 8: Print titles of first 50 songs
            i=1
            #clear the listbox
            lbxResult.delete(0, tk.END)
            #for every song in the similar songs list
            for song in sorted_similar_songs:
                #insert position, song info and separator
                lbxResult.insert(tk.END, "Song Position:"+str(i))
                lbxResult.insert(tk.END,"Song ID: "+ ids[i])
                lbxResult.insert(tk.END,"Song Name: "+names[i])
                lbxResult.insert(tk.END,"Artist: "+artists[i])
                lbxResult.insert(tk.END,"Genre: "+genres[i])
                lbxResult.insert(tk.END,"Duration: "+durations[i])
                lbxResult.insert(tk.END,"Date Released: "+releases[i])
                lbxResult.insert(tk.END,"----------------------------------")

                i=i+1
                if i>50:
                    break
        #button to trigger recommendation
        btnMakeRecommendation = tk.Button(self, text="Make Recommendation", font=LARGE_FONT, command=recommendEmotionArtistG)
        btnMakeRecommendation.pack()

        #button to go back to the Emotion recommendation menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(EmotionMenu))
        btnBack.pack() 
        

#form used to edit emotions        
class EditEmotionsForm(tk.Frame):
    
   #initialise form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #select all songs function
        def select_songse(con):
            #create cursor
            cursor1=con.cursor()
            #execute the query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions")
            #store result
            row=cursor1.fetchall()
            #open csv file
            with open('songsEmotions.csv', 'w', newline= '') as f:
                #say we are writing to the files
                a = csv.writer(f, delimiter=',')
                #write the headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write the results as row
                a.writerows(row)
        #select songs
        select_songse(connection)
        #initialse variables
        songID=""
        songName=""
        songArtist=""
        releaseYear=""
        releaseMonth=""
        releaseDay=""
        songGenre=""
        songDuration=""
        songEmotion=""
        
        #label for title
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: EDIT SONGS WITH EMOTIONS", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #listbox for the result
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        #label for song id
        lblSongID = tk.Label(self, text="Insert ID of song to edit: ", font=LARGE_FONT)
        lblSongID.pack()
        #input for song id
        txtSongID=tk.Entry(self, textvariable=songID, font=LARGE_FONT)
        txtSongID.pack()
        #label for song name
        lblSongName = tk.Label(self, text="Insert New Song Name : ", font=LARGE_FONT)
        lblSongName.pack()
        #input for song name
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        #label for song artist
        lblSongArtist = tk.Label(self, text="Insert New Song Artist : ", font=LARGE_FONT)
        lblSongArtist.pack()
        #input for song artist
        txtSongArtist=tk.Entry(self, textvariable=songArtist, font=LARGE_FONT)
        txtSongArtist.pack()
        #label for release year
        lblReleaseYear = tk.Label(self, text="Insert New Release Year : ", font=LARGE_FONT)
        lblReleaseYear.pack()
        #input for release year
        txtReleaseYear=tk.Entry(self, textvariable=releaseYear, font=LARGE_FONT)
        txtReleaseYear.pack()
        #label for release month
        lblReleaseMonth = tk.Label(self, text="Insert New Release Month : ", font=LARGE_FONT)
        lblReleaseMonth.pack()
        #input for release month
        txtReleaseMonth=tk.Entry(self, textvariable=releaseMonth, font=LARGE_FONT)
        txtReleaseMonth.pack()
        #label for release day
        lblReleaseDay = tk.Label(self, text="Insert New Release Day : ", font=LARGE_FONT)
        lblReleaseDay.pack()
        #input for release day
        txtReleaseDay=tk.Entry(self, textvariable=releaseDay, font=LARGE_FONT)
        txtReleaseDay.pack()
        #label  for song genre
        lblSongGenre = tk.Label(self, text="Insert New Song Genre : ", font=LARGE_FONT)
        lblSongGenre.pack()
        #input for song genre
        txtSongGenre=tk.Entry(self, textvariable=songGenre, font=LARGE_FONT)
        txtSongGenre.pack()
        #label for song duration
        lblSongDuration = tk.Label(self, text="Insert New Song Duration : ", font=LARGE_FONT)
        lblSongDuration.pack()
        #input for song duration
        spnSongDuration=tk.Spinbox(self,from_=0, to=10000, textvariable=songDuration, font=LARGE_FONT)
        spnSongDuration.pack()
        #label for song emotion
        lblSongEmotion = tk.Label(self, text="Insert New Song Emotion : ", font=LARGE_FONT)
        lblSongEmotion.pack()
        #input for song emotion
        txtSongEmotion=tk.Entry(self, textvariable=songEmotion, font=LARGE_FONT)
        txtSongEmotion.pack()
        #functions required 
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]

        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        def get_emotion_from_index(index):
            df["emotion"]=df["emotion"].astype(str)
            return df[df.song_id == index]["emotion"].values[0]
        
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songsEmotions.csv")
        #convert to list
        songs=df.values.tolist()
        #for every song in the list
        for song in songs:
            #insert song info and separator
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"Assigned Emotion: "+get_emotion_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        
        
        
        #function to edit the emotions
        def editEmotionG():
            #select the song
            select_songse(connection)
            ###### helper functions. Use them when needed #######
            #required functions
            def get_name_from_index(index):
                df['song_name']=df['song_name'].astype(str)
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                df['artist']=df['artist'].astype(str)
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                df['song_genre']=df['song_genre'].astype(str)
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                df['date_released']=df['date_released'].astype(str)
                return df[df.song_id == index]["date_released"].values[0]
            
            def get_emotion_from_index(index):
                df["emotion"]=df["emotion"].astype(str)
                return df[df.song_id == index]["emotion"].values[0]
            
            def get_index_from_name(title):
                return df[df.song_name == title]["song_id"].values[0]
            #set the variables to user inputs
            songID=txtSongID.get()
            songArtist=txtSongArtist.get()
            releaseYear=txtReleaseYear.get()
            releaseMonth=txtReleaseMonth.get()
            releaseDay=txtReleaseDay.get()
            releaeDate=releaseYear+"-"+releaseMonth+"-"+releaseDay
            songGenre=txtSongGenre.get()
            songDuration=spnSongDuration.get().astype(str)
            songEmotion=txtSongEmotion.get()
            songName=txtSongName.get()

            ##################################################
            ##Step 1: Read CSV File
            df = pd.read_csv("songsEmotionsSingle.csv")
            #set blank variable for queries
            songInfo=""
            songInfo2=""
            
            if(songID==""):
                songID="-1"
            if(songID!="-1"):
                if(songName==""):
                    songName=get_name_from_index(songID)
                if(songArtist==""):
                    songArtist=get_artist_from_index(songID)    
                if(releaseDate==""+"-"+""+"-"+"" 
                   or releaseDate==releaseYear+"-"+releaseMonth +"-"+""
                   or releaseDate==releaseYear+"-"+""+"-"+releaseDay
                   or releaseDate==""+"-"+releaseMonth+"-"+releaseDay):
                    releaseDate=get_released_from_index(songID)
                if(songDuration==""):
                    songDuration=get_duration_from_index(songID)
                if(songGenre==""):
                    songGenre=get_genre_from_index(songID)
                if(songEmotion==""):
                    songEmotion=get_emotion_from_index(songID)
                #update the songs emotions file file
                songInfo="UPDATE songs_emotions SET song_name=\""+songName+"\", atist=\""+songArtist+"\", date_released=\""+releaseDate+"\", song_length_secs="+songDuration+", song_genre=\""+songGenre+"\", song_emotion=\""+songEmotion+"\" WHERE song_id="+songID
                #update songs variable
                songInfo2="UPDATE songs SET song_name=\""+songName+"\", atist=\""+songArtist+"\", date_released=\""+releaseDate+"\", song_length_secs="+songDuration+", song_genre=\""+songGenre+"\" WHERE song_id="+songID
            #execute the query to update songs emotions table
            cursor1.execute(songInfo)
            #execute query to update songs table
            cursor1.execute(songInfo2)
            #commit, so that it actually does the above
            connection.commit()
           
        #button to trigger the edit function
        btnEdit = tk.Button(self, text="Edit Song", font=LARGE_FONT, command=editEmotionG)
        btnEdit.pack()
        
        #button to go back to edit menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(EditMenu))
        btnBack.pack() 
        
        
#view form, where user can view all records        
class ViewForm(tk.Frame):
    #initialise the form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #select the songs
        def select_songse(con):
            #create cursor
            cursor1=con.cursor()
            #slect song info via query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions")
            #stor query result into variable
            row=cursor1.fetchall()
            #open songs emotion file
            with open('songsEmotions.csv', 'w', newline= '') as f:
                #writing to the file
                a = csv.writer(f, delimiter=',')
                #write headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #wrie the rows
                a.writerows(row)
        #select songs
        select_songse(connection)
        #functions needed to get song info from the dataframe
        def get_name_from_index(index):
                df['song_name']=df['song_name'].astype(str)
                return df[df.song_id == index]["song_name"].values[0]

        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]

        def get_emotion_from_index(index):
            df["emotion"]=df["emotion"].astype(str)
            return df[df.song_id == index]["emotion"].values[0]

        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File

        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        df = pd.read_csv("songsEmotions.csv")
        #convert to list
        songs=df.values.tolist()
        #for every song in list
        for song in songs:
            #insert song info
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"A-ssigned Emotion: "+get_emotion_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")

        #funvtion to refrsh the listbox
        def refresh():
            def get_name_from_index(index):
                df['song_name']=df['song_name'].astype(str)
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                df['artist']=df['artist'].astype(str)
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                df['song_genre']=df['song_genre'].astype(str)
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                df['date_released']=df['date_released'].astype(str)
                return df[df.song_id == index]["date_released"].values[0]

            def get_emotion_from_index(index):
                df["emotion"]=df["emotion"].astype(str)
                return df[df.song_id == index]["emotion"].values[0]

            def get_index_from_name(title):
                return df[df.song_name == title]["song_id"].values[0]
            df = pd.read_csv("songsEmotions.csv")
            #convert dataframe
            songs=df.values.tolist()
            lbxResult.delete(0, tk.END)
            #same code as earlier on in this form
            for song in songs:
                #listbox with result
                lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
                lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
                lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
                lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
                lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
                lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
                lbxResult.insert(tk.END,"Assigned Emotion: "+get_emotion_from_index(song[0]))
                lbxResult.insert(tk.END,"----------------------------------")
        #button to trigger refreshfunction
        btnRefresh = tk.Button(self, text="Refresh", font=LARGE_FONT, command=refresh)
        btnRefresh.pack() 
        #button to go back to main menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(MainMenu))
        btnBack.pack() 
        
#form used to edit        
class EditForm(tk.Frame):
    #initialise
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #function to select songs
        def select_songs(con):
            #create cursor
            cursor1=con.cursor()
            #execute select query
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs")
            #store result
            row=cursor1.fetchall()
            #open songs file
            with open('songs.csv', 'w', newline= '') as f:
                #writing to file
                a = csv.writer(f, delimiter=',')
                #write headings
                a.writerow(["song_id", "song_name", "artist","date_released","song_genre","song_length_secs", "emotion"])
                #write records
                a.writerows(row)
        #select songs
        select_songs(connection)
        #initialise
        songID=""
        songName=""
        songArtist=""
        releaseYear=""
        releaseMonth=""
        releaseDay=""
        releaseDate=""
        songGenre=""
        songDuration=""
        songEmotion=""
        songName=""
        
        #label for title
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: EDIT SONGS WITH EMOTIONS", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #listbox for result
        lbxResult=tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        #label for song id
        lblSongID = tk.Label(self, text="Insert ID of song to edit: ", font=LARGE_FONT)
        lblSongID.pack()
        #input for song id
        txtSongID=tk.Entry(self, textvariable=songID, font=LARGE_FONT)
        txtSongID.pack()
        #label for song name
        lblSongName = tk.Label(self, text="Insert New Song Name : ", font=LARGE_FONT)
        lblSongName.pack()
        #input for song name
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        #label for song artist
        lblSongArtist = tk.Label(self, text="Insert New Song Artist : ", font=LARGE_FONT)
        lblSongArtist.pack()
        #input for song artist
        txtSongArtist=tk.Entry(self, textvariable=songArtist, font=LARGE_FONT)
        txtSongArtist.pack()
        #label for release year of song
        lblReleaseYear = tk.Label(self, text="Insert New Release Year : ", font=LARGE_FONT)
        lblReleaseYear.pack()
        #input for release year of song
        txtReleaseYear=tk.Entry(self, textvariable=releaseYear, font=LARGE_FONT)
        txtReleaseYear.pack()
        #label for release month of song
        lblReleaseMonth = tk.Label(self, text="Insert New Release Month : ", font=LARGE_FONT)
        lblReleaseMonth.pack()
        #input for release month
        txtReleaseMonth=tk.Entry(self, textvariable=releaseMonth, font=LARGE_FONT)
        txtReleaseMonth.pack()
        #label for release day of song
        lblReleaseDay = tk.Label(self, text="Insert New Release Day : ", font=LARGE_FONT)
        lblReleaseDay.pack()
        #input for release day of song
        txtReleaseDay=tk.Entry(self, textvariable=releaseDay, font=LARGE_FONT)
        txtReleaseDay.pack()
        #label for genre of song
        lblSongGenre = tk.Label(self, text="Insert New Song Genre : ", font=LARGE_FONT)
        lblSongGenre.pack()
        #input for genre of song
        txtSongGenre=tk.Entry(self, textvariable=songGenre, font=LARGE_FONT)
        txtSongGenre.pack()
        #label of duration for song
        lblSongDuration = tk.Label(self, text="Insert New Song Duration : ", font=LARGE_FONT)
        lblSongDuration.pack()
        #input for song duration in seconds
        spnSongDuration=tk.Spinbox(self,from_=0, to=10000, textvariable=songDuration, font=LARGE_FONT)
        spnSongDuration.pack()
        #label for song emotion
        lblSongEmotion = tk.Label(self, text="Insert New Song Emotion : ", font=LARGE_FONT)
        lblSongEmotion.pack()
        #input for assigning/assigned emotion
        txtSongEmotion=tk.Entry(self, textvariable=songEmotion, font=LARGE_FONT)
        txtSongEmotion.pack()
        #functions to get items from dataframe
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]

        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]
        
        
        
        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        ##################################################
        ##Step 1: Read CSV File
        df = pd.read_csv("songs.csv")
        #convert songs to list
        songs=df.values.tolist()
        #for every item in the list
        for song in songs:
            #insert song info
            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        
        
        
        #edit function
        def editG():
            #select the songs
            select_songs(connection)
            cursor1=connection.cursor()
            ###### helper functions. Use them when needed #######
            def get_name_from_index(index):
                df['song_name']=df['song_name'].astype(str)
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                df['artist']=df['artist'].astype(str)
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                df['song_genre']=df['song_genre'].astype(str)
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                df['date_released']=df['date_released'].astype(str)
                return df[df.song_id == index]["date_released"].values[0]
            
            def get_index_from_name(title):
                return df[df.song_name == title]["song_id"].values[0]
            #set to user inputs
            songID=txtSongID.get()
            songArtist=txtSongArtist.get()
            releaseYear=txtReleaseYear.get()
            releaseMonth=txtReleaseMonth.get()
            releaseDay=txtReleaseDay.get()
            releaseDate=releaseYear+"-"+releaseMonth+"-"+releaseDay
            songGenre=txtSongGenre.get()
            songDuration=spnSongDuration.get()
            songEmotion=txtSongEmotion.get()
            songName=txtSongName.get()

            ##################################################
            ##Step 1: Read CSV File
            df = pd.read_csv("songs.csv")
            #query variables
            songInfo=""
            songInfo2=""
            #make sure song id is not blank
            if(songID==""):
                songID="-1"
            if(songID!="-1"):
                if(songName==""):
                    songName=get_name_from_index(int(songID))
                if(songArtist==""):
                    songArtist=get_artist_from_index(int(songID))    
                if(releaseDate==""+"-"+""+"-"+"" 
                   or releaseDate==releaseYear+"-"+releaseMonth +"-"+""
                   or releaseDate==releaseYear+"-"+""+"-"+releaseDay
                   or releaseDate==""+"-"+releaseMonth+"-"+releaseDay):
                    releaseDate=get_released_from_index(int(songID))
                if(songDuration==""):
                    songDuration=get_duration_from_index(int(songID))
                if(songGenre==""):
                    songGenre=get_genre_from_index(int(songID))
            #update sons_emotion table    
            songInfo="UPDATE songs_emotions SET song_name=%s, artist=%s, date_released=%s, song_length_secs=%s, song_genre=%s, emotion=%s WHERE song_id=%s"
            val=(songName, songArtist, releaseDate, songDuration, songGenre, songEmotion, songID)
            #update songs table
            songInfo2="UPDATE songs SET song_name=%s, artist=%s, date_released=%s, song_length_secs=%s, song_genre=%s WHERE song_id=%s"
            val2=(songName, songArtist, releaseDate, songDuration, songGenre, songID)
            #execute first query
            cursor1.execute(songInfo, val)
            #execute second query
            cursor1.execute(songInfo2, val2)
            #commit to executions
            connection.commit()
           
        #button to execute edit function
        btnEdit = tk.Button(self, text="Edit Song", font=LARGE_FONT, command=editG)
        btnEdit.pack()

        #button to go back to main menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(EditMenu))
        btnBack.pack() 
        

                
#form used to assign emotions        
class AssignEmotionForm(tk.Frame):

    #initialise forms
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #initialise required variables
        songID=""
        songEmotion=""
        
        #these help to get required parts of database
        def get_name_from_index(index):
            df['song_name']=df['song_name'].astype(str)
            return df[df.song_id == index]["song_name"].values[0]

        def get_artist_from_index(index):
            df['artist']=df['artist'].astype(str)
            return df[df.song_id == index]["artist"].values[0]

        def get_genre_from_index(index):
            df['song_genre']=df['song_genre'].astype(str)
            return df[df.song_id == index]["song_genre"].values[0]

        def get_duration_from_index(index):
            df["song_length_secs"]=df["song_length_secs"].astype(str)
            return df[df.song_id == index]["song_length_secs"].values[0]

        def get_released_from_index(index):
            df['date_released']=df['date_released'].astype(str)
            return df[df.song_id == index]["date_released"].values[0]

        def get_index_from_name(title):
            return df[df.song_name == title]["song_id"].values[0]

        def get_emotion_from_index(index):
            df["emotion"]=df["emotion"].astype(str)
            return df[df.song_id == index]["emotion"].values[0]
       
        
        #label for title
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: ASSIGN EMOTION", font=EXTRA_LARGE_FONT)
        lblTitle.pack()
        #label for song id
        lblSongID = tk.Label(self, text="Insert Song ID: ", font=LARGE_FONT)
        lblSongID.pack()
        #input for song id
        txtSongID=tk.Entry(self, font=LARGE_FONT)
        txtSongID.pack()
        #label for song emotion
        lblSongEmotion = tk.Label(self, text="Emotion To Assign: ", font=LARGE_FONT)
        lblSongEmotion.pack()
        #input for song emotion
        txtSongEmotion=tk.Entry(self, font=LARGE_FONT)
        txtSongEmotion.pack()
        
        #function to assign emotion
        def assignEmotionG():
            ###### helper functions. Use them when needed #######
            def get_name_from_index(index):
                df['song_name']=df['song_name'].astype(str)
                return df[df.song_id == index]["song_name"].values[0]

            def get_artist_from_index(index):
                df['artist']=df['artist'].astype(str)
                return df[df.song_id == index]["artist"].values[0]

            def get_genre_from_index(index):
                df['song_genre']=df['song_genre'].astype(str)
                return df[df.song_id == index]["song_genre"].values[0]

            def get_duration_from_index(index):
                df["song_length_secs"]=df["song_length_secs"].astype(str)
                return df[df.song_id == index]["song_length_secs"].values[0]

            def get_released_from_index(index):
                df['date_released']=df['date_released'].astype(str)
                return df[df.song_id == index]["date_released"].values[0]

            def get_index_from_name(title):
                return df[df.song_name == title]["song_id"].values[0]
            
            def get_emotion_from_index(index):
                df["emotion"]=df["emotion"].astype(str)
                return df[df.song_id == index]["emotion"].values[0]
            #create cursor
            cursor1=connection.cursor()
            #set required variables to user input
            songemotion=txtSongEmotion.get()
            songid=txtSongID.get()
            if(songID==""):
                songID="-1"
            if(songID!="-1"):
                #update sons_emotion table to assign emotion
                songInfo="UPDATE songs_emotions SET emotion=\""+songemotion+"\" WHERE song_id="+songID
            #execute query
            cursor1.execute(songInfo)
            #commit
            connection.commit()
            #clear user input fields
            txtSongEmotion.delete(0,tk.END)
            txtSongID.delete(0, tk.END)
        #declair listbox
        lbxResult = tk.Listbox(self,width=50, font=LARGE_FONT)
        lbxResult.pack()
        #open songs file
        df = pd.read_csv("songs.csv")
        songs=df.values.tolist()
        for song in songs:

            lbxResult.insert(tk.END,"Song ID: "+str(song[0]))
            lbxResult.insert(tk.END,"Song Name: "+get_name_from_index(song[0]))
            lbxResult.insert(tk.END,"Artist: "+get_artist_from_index(song[0]))
            lbxResult.insert(tk.END,"Genre: "+get_genre_from_index(song[0]))
            lbxResult.insert(tk.END,"Duration: "+get_duration_from_index(song[0]))
            lbxResult.insert(tk.END,"Date Released: "+get_released_from_index(song[0]))
            lbxResult.insert(tk.END,"----------------------------------")
        #button to assign emotions
        btnAssignEmotion = tk.Button(self, text="Assign Emotion", font=LARGE_FONT, command = assignEmotionG)
        btnAssignEmotion.pack()
        
       
        #button to go back to edit menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(EditMenu))
        btnBack.pack()  
        
        
#button to insert with emotions        
class InsertEmotionForm(tk.Frame):
    #initialise forms
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #the function to select songs
        def select_songse(con):
            #create the cursor for queries
            cursor1=con.cursor()
            #execute the queries
            cursor1.execute("SELECT song_name, artist, date_released, song_genre, song_length_secs, emotion FROM songs_emotions")
            #store result in variable
            row=cursor1.fetchall()
            
            with open('songsEmotions.csv', 'w', newline= '') as f:
                #write to files
                a = csv.writer(f, delimiter=',')
                #wrie headings
                a.writerow(["song_name", "artist","date_released","song_genre","song_length_secs"])
                #write rows needed to file
                a.writerows(row)
        #initialise variables
        songName=""
        songArtist=""
        songReleaseYear=""
        songReleaseMonth=""
        songReleaseDay=""
        songGenre=""
        songLength=""
        songDuration=""
        songEmotion=""
        
        
        
        #label for title of application
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: INSERT NEW SONG AND ASSIGN EMOTION", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #label for name of the song: prompts user to insert song name
        lblSongName=tk.Label(self, text="Insert Song Name", font=LARGE_FONT)
        lblSongName.pack()
        #input for name of the song
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        #label for song artist 
        lblArtist=tk.Label(self, text="Insert Song Artist", font=LARGE_FONT)
        lblArtist.pack()
        #input for artist of song
        txtSongArtist=tk.Entry(self, textvariable=songArtist, font=LARGE_FONT)
        txtSongArtist.pack()
        #label for release year prompt
        lblYearReleased=tk.Label(self, text="Insert Year Released In Format yyyy:", font=LARGE_FONT)
        lblYearReleased.pack()
        #input for release year of song
        txtYearReleased=tk.Entry(self, textvariable=songReleaseYear, font=LARGE_FONT)
        txtYearReleased.pack()
        #label for release month
        lblMonthReleased=tk.Label(self, text="Insert Month Released In Format mm:", font=LARGE_FONT)
        lblMonthReleased.pack()
        #input for release month
        txtMonthReleased=tk.Entry(self, textvariable=songReleaseMonth, font=LARGE_FONT)
        txtMonthReleased.pack()
        #label for release day
        lblDayReleased=tk.Label(self, text="Insert Day Released In Format dd", font=LARGE_FONT)
        lblDayReleased.pack()
        #input for release day
        txtDayReleased=tk.Entry(self, textvariable=songReleaseDay, font=LARGE_FONT)
        txtDayReleased.pack()
        #label for genre of song
        lblSongGenre=tk.Label(self, text="Insert Song Genre", font=LARGE_FONT)
        lblSongGenre.pack()
        #input for genre of song
        txtSongGenre=tk.Entry(self, textvariable=songGenre, font=LARGE_FONT)
        txtSongGenre.pack()
        #label for duration of the song
        lblSongDuration=tk.Label(self, text="Insert Song Duration In Seconds", font=LARGE_FONT)
        lblSongDuration.pack()
        #input for song, as spinbox
        spnSongDuration = tk.Spinbox(self, from_= 1, to_=10000, textvariable=songLength)  
        spnSongDuration.pack()  
        #label for song emotion
        lblSongEmotion=tk.Label(self, text="Assign Emotion To Song", font=LARGE_FONT)
        lblSongEmotion.pack()
        #input for song emotion
        txtSongEmotion=tk.Entry(self, textvariable=songEmotion, font=LARGE_FONT)
        txtSongEmotion.pack()
        
        
        
        #function  to insert data
        def insertDataEmotionG():
            try:
                #create cursor so we can execute queries
                cursor1=connection.cursor()
                #set default values
                if(songName==""):
                    songName="unknown"
                songArtist=txtSongArtist.get()
                if(songArtist==""):
                    songArtist="unknown"
                songReleaseYear=txtYearReleased.get()
                if(songReleaseYear==""):
                    songReleaseYear="1899"
                songReleaseMonth=txtMonthReleased.get()
                if(songReleaseYear==""):
                    songReleaseMonth="01"
                songReleaseDay=txtDayReleased.get()
                if(songReleaseDay==""):
                    songReleaseDay="01"
                releaseDate=songReleaseYear+"-"+songReleaseMonth+"-"+songReleaseDay
                songGenre=txtSongGenre.get()
                if(songGenre==""):
                    songReleaseMonth="unknown"
                songDuration=spnSongDuration.get()
                songEmotion=txtSongEmotion.get()
                #the insert queries, inserting into 2 different tables
                songeInfo="INSERT into songs_emotions(song_name, artist, date_released, song_genre, song_length_secs, emotion) values(\""+songName+"\", \""+songArtist+"\", \""+releaseDate+"\", \""+songGenre+"\", "+songDuration+", \""+songEmotion+"\" )"
                songInfo="INSERT INTO songs(song_name, artist, date_released, song_genre, song_length_secs) values(\""+songName+"\", \""+songArtist+"\", \""+releaseDate+"\", \""+songGenre+"\", "+songDuration+" )"
                #execute queries
                cursor1.execute(songeInfo)
                cursor1.execute(songInfo)
                #commit to database changes
                connection.commit()
                #clear fields
                txtSongName.delete(0, tk.END)
                txtSongArtist.delete(0, tk.END)
                txtSongReleaseYear.delete(0, tk.END)
                
                txtSongReleaseMonth.delete(0, tk.END)
                txtSongReleaseDay.delete(0, tk.END)
                txtSongGenre.delete(0, tk.END)
                spnSongDuration.delete(0,tk.END)
                txtSongEmotion.delete(0, tk.END)
                
                sleep(1)
                #select songs
                select_songse(connection)
            except Exception as e:
                print("Exeception occured:{}".format(e))

        #button to trigger insert function
        btnInsert = tk.Button(self, text="Insert Song", font=LARGE_FONT, command=insertDataEmotionG)
        btnInsert.pack()
        #button to go back to the insert menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(InsertMenu))
        btnBack.pack()            
#form to insert data without emotions
class InsertForm(tk.Frame):

    #initialsise form items
    def __init__(self, parent, controller):
        
        
        tk.Frame.__init__(self,parent)
        #select songs
        def select_songsG(con):
            
            #create cursor for executing queries
            cursor1=con.cursor()
            #execute select statement
            cursor1.execute("SELECT song_id, song_name, artist, date_released, song_genre, song_length_secs FROM songs")
            #store result in variable
            row=cursor1.fetchall()
            #open songs file
            with open('songs.csv', 'w', newline= '') as f:
                #say we are writing to file
                a = csv.writer(f, delimiter=',')
                #insert headings
                header=["song_id", "song_name", "artist", "date_released", "song_genre", "song_length_secs"]
                #write the header row
                a.writerow(header)
                #write the rows of the song data
                a.writerows(row)
        #initialise required variables
        songName=""
        songArtist=""
        songReleaseYear=""
        songReleaseMonth=""
        songReleaseDay=""
        songGenre=""
        songLength=""
        songDuration=""
        songEmotion=""
        
        
        #title of application and form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: INSERT NEW SONG", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #prompt for inserting song name
        lblSongName=tk.Label(self, text="Insert Song Name", font=LARGE_FONT)
        lblSongName.pack()
        #input for song name
        txtSongName=tk.Entry(self, textvariable=songName, font=LARGE_FONT)
        txtSongName.pack()
        #prompt for inserting song artist
        lblArtist=tk.Label(self, text="Insert Song Artist", font=LARGE_FONT)
        lblArtist.pack()
        #input for song artist
        txtSongArtist=tk.Entry(self, textvariable=songArtist, font=LARGE_FONT)
        txtSongArtist.pack()
        #prompt for song release year
        lblYearReleased=tk.Label(self, text="Insert Year Released In Format yyyy:", font=LARGE_FONT)
        lblYearReleased.pack()
        #input for song release year
        txtYearReleased=tk.Entry(self, textvariable=songReleaseYear, font=LARGE_FONT)
        txtYearReleased.pack()
        #prompt for inserting song release month
        lblMonthReleased=tk.Label(self, text="Insert Month Released In Format mm:", font=LARGE_FONT)
        lblMonthReleased.pack()
        #input for song release month
        txtMonthReleased=tk.Entry(self, textvariable=songReleaseMonth, font=LARGE_FONT)
        txtMonthReleased.pack()
        #prompt for inserting song release day
        lblDayReleased=tk.Label(self, text="Insert Day Released In Format dd", font=LARGE_FONT)
        lblDayReleased.pack()
        #input for song release day
        txtDayReleased=tk.Entry(self, textvariable=songReleaseDay, font=LARGE_FONT)
        txtDayReleased.pack()
        #prompt for inserting song genre
        lblSongGenre=tk.Label(self, text="Insert Song Genre", font=LARGE_FONT)
        lblSongGenre.pack()
        #input for song genre
        txtSongGenre=tk.Entry(self, textvariable=songGenre, font=LARGE_FONT)
        txtSongGenre.pack()
        #prompt for inserting song duration in seconds
        lblSongDuration=tk.Label(self, text="Insert Song Duration In Seconds", font=LARGE_FONT)
        lblSongDuration.pack()
        #input for song duration in seconds
        spnSongDuration = tk.Spinbox(self, from_= 1, to_=10000,textvariable=songLength)  
        spnSongDuration.pack()
        #label for error message
        lblError = tk.Label(self, text="", font=LARGE_FONT)
        lblError.pack
        
        #function for inserting song data
        def insertDataG():
            try:
                #cursor for executing queries
                cursor1=connection.cursor()
                #set song name to user input
                songName=txtSongName.get()
                #if blank, change song name to unknown. basically setting default values
                if(songName==""):
                    songName="unknown"
                songArtist=txtSongArtist.get()
                if(songArtist==""):
                    songArtist="unknown"
                songReleaseYear=txtYearReleased.get()
                if(songReleaseYear==""):
                    songReleaseYear="1899"
                songReleaseMonth=txtMonthReleased.get()
                if(songReleaseYear==""):
                    songReleaseMonth="01"
                songReleaseDay=txtDayReleased.get()
                if(songReleaseDay==""):
                    songReleaseDay="01"
                releaseDate=songReleaseYear+"-"+songReleaseMonth+"-"+songReleaseDay
                songGenre=txtSongGenre.get()
                if(songGenre==""):
                    songGenre="unknown"
                songDuration=spnSongDuration.get()
                #queries for inserting data
                songInfo="INSERT INTO songs(song_name, artist, date_released, song_genre, song_length_secs) values(\""+songName+"\", \""+songArtist+"\", \""+releaseDate+"\", \""+songGenre+"\", "+songDuration+" )"
                songInfo2="INSERT INTO songs_emotions(song_name, artist, date_released, song_genre, song_length_secs) values(\""+songName+"\", \""+songArtist+"\", \""+releaseDate+"\", \""+songGenre+"\", "+songDuration+" )"
                #execute queries
                cursor1.execute(songInfo)
                cursor1.execute(songInfo2)
                #commit changes to database
                connection.commit()
                #clear all fields
                txtSongName.delete(0, tk.END)
                txtSongArtist.delete(0, tk.END)
                txtYearReleased.delete(0, tk.END)
                
                txtMonthReleased.delete(0, tk.END)
                txtDayReleased.delete(0, tk.END)
                txtSongGenre.delete(0, tk.END)
                spnSongDuration.delete(0,tk.END)
                select_songsG(connection)
                
            except Exception as e:
                lblError.text=("Exeception occured:{}".format(e))
        #button to trigger insert function
        btnInsert = tk.Button(self, text="Insert Song", font=LARGE_FONT, command=insertDataG)
        btnInsert.pack()
        #button to go back to insert menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(InsertMenu))
        btnBack.pack()   

                            
                            
#main menu of application
class MainMenu(tk.Frame):
    #initialise form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #application title    
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #button to go to the Insert/Edit menu
        btnInsertEditMenu = tk.Button(self, text="Insert/Edit Song Data", font=LARGE_FONT, command=lambda: controller.show_frame(InsertEditMenu))
        btnInsertEditMenu.pack()
        #button to go to the main menu of the Recommendations
        btnRecommendationMenu = tk.Button(self, text="Make Recommendation", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendationMainMenu))
        btnRecommendationMenu.pack()
        #button to go to form that allows user to view all songs
        btnView =  tk.Button(self, text="View", font=LARGE_FONT, command=lambda: controller.show_frame(ViewForm))
        btnView.pack()
        
#main menu of the recommendations        
class RecommendationMainMenu(tk.Frame):
    #initialise form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #title of application and menu    
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: MAIN RECOMMENDATION MENU", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #button to go to the menu for recommendations without emotions
        btnRecommendation = tk.Button(self, text="Make Recommendation Without Emotions", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendationMenu))
        btnRecommendation.pack()
        #button to go to menu for recommendations with emotions
        btnRecommendationE = tk.Button(self, text="Make Recommendation with Emotions", font=LARGE_FONT, command=lambda: controller.show_frame(EmotionMenu))
        btnRecommendationE.pack()
        #button to go back to main menu
        btnBack =  tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(MainMenu))
        btnBack.pack()
        
#form for the insert/edit menu        
class InsertEditMenu(tk.Frame):
    
    #initialise form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        
        
        #title of application and form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: INSERT/EDIT MENU", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #button to go to the insert menu
        btnInsert = tk.Button(self, text="Insert Song Data", font=LARGE_FONT, command=lambda: controller.show_frame(InsertMenu))
        btnInsert.pack()
        #button to go to the edit menu
        btnEdit = tk.Button(self, text="Edit Song Data", font=LARGE_FONT, command=lambda: controller.show_frame(EditMenu))
        btnEdit.pack()
        #button to go back to the main menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(MainMenu))
        btnBack.pack()
        
        
class InsertMenu(tk.Frame):
    
                            
    #initialise form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #title of application and form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: INSERT MENU", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #button to go to the insert form
        btnInsert = tk.Button(self, text="Insert Song Data", font=LARGE_FONT, command=lambda: controller.show_frame(InsertForm))
        btnInsert.pack()
        #button to go to the insert emotions form
        btnInsertAssign = tk.Button(self, text="Insert Song Data And Assign Emotion", font=LARGE_FONT, command=lambda: controller.show_frame(InsertEmotionForm))
        btnInsertAssign.pack()
        #button to go back to the insert/edit menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(InsertEditMenu))
        btnBack.pack()
#form for the edit menu        
class EditMenu(tk.Frame):
    
    #initialise form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        
        #title of application and form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: EDIT MENU", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #button to go to the assign emotion form
        btnAssign = tk.Button(self, text="Assign Emotion", font=LARGE_FONT, command=lambda: controller.show_frame(AssignEmotionForm))
        btnAssign.pack()
        #button to edit songs without emotions
        btnEditN = tk.Button(self, text="Edit Without Emotion", font=LARGE_FONT, command=lambda: controller.show_frame(EditForm))
        btnEditN.pack()
        #button to edit songs with emotions
        btnEditE = tk.Button(self, text="Edit With Emotion", font=LARGE_FONT, command=lambda: controller.show_frame(EditEmotionsForm))
        btnEditE.pack()
        #button to go back to the insert/edit menu
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(InsertEditMenu))
        btnBack.pack()

#form for normal recommendations                 
class RecommendationMenu(tk.Frame):
    
    #initialise the form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        #title of application and form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: RECOMMENDATION MENU", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #button to make recommendation based on song name + artist
        btnArtist = tk.Button(self, text="Artist", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendArtistForm))
        btnArtist.pack()
        #button to make recommendation based on song name + duration
        btnDuration = tk.Button(self, text="Song Duration", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendDurationForm))
        btnDuration.pack()
        #button to make recommendation based on song name + genre
        btnGenre = tk.Button(self, text="Genre", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendGenreForm))
        btnGenre.pack()
        #button to make recommendation based on song name + release date
        btnReleased = tk.Button(self, text="Date Released", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendReleasedForm))
        btnReleased.pack()
        #button to go back to main menu for recommendations
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendationMainMenu))
        btnBack.pack()
#recommendation menu for songs with emotions                 
class EmotionMenu(tk.Frame):
    
                            
    #initialise form
    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        
        
        #title of application and form
        lblTitle = tk.Label(self, text="MD RECOMMENDATION SYSTEM: RECOMMENDATION MENU", font=EXTRA_LARGE_FONT)
        lblTitle.pack(pady=10,padx=10)
        #button to make recommendation based on song name + emotion + artist
        btnEmotionArtist = tk.Button(self, text="Emotion With Artist", font=LARGE_FONT, command=lambda: controller.show_frame(RecEArtistForm))
        btnEmotionArtist.pack()
        #button to make recommendation based on song name + emotion + duration
        btnEmotionDuration = tk.Button(self, text="Emotion With Song Duration", font=LARGE_FONT, command=lambda: controller.show_frame(RecEDurationForm))
        btnEmotionDuration.pack()
        #button to make recommendation based on song name + emotion + genre
        btnEmotionGenre = tk.Button(self, text="Emotion With Genre", font=LARGE_FONT, command=lambda: controller.show_frame(RecEGenreForm))
        btnEmotionGenre.pack()
        #button to make recommendation based on song name + emotion + release date
        btnEmotionReleased = tk.Button(self, text="Emotion With Date Released", font=LARGE_FONT, command=lambda: controller.show_frame(RecEReleasedForm))
        btnEmotionReleased.pack()
        #button to go back to the main menu for recommendations
        btnBack = tk.Button(self, text="Back", font=LARGE_FONT, command=lambda: controller.show_frame(RecommendationMainMenu))
        btnBack.pack()
#start application
app = MyApplication()
app.mainloop()